In [ ]:
import os
import re
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from datetime import datetime
from src.utils.deepseek_utils import prompt_deepseek
from loguru import logger
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
from loguru import logger
import pandas as pd


embeddings = OpenAIEmbeddings(model="text-embedding-3-small")


def embed_manifestos(party: str) -> dict[str, Chroma]:
    """
    Embeds the manifestos for a given party, returns them and saves them in a vector store.
    """
    logger.info(f"Embedding manifestos for {party}")
    vs_root = "data/vectorstores"
    os.makedirs(vs_root, exist_ok=True)
    splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=100)

    manifesto_dir = "data/text/manifestos_cleaned"
    manifesto_paths = [
        os.path.join(manifesto_dir, p)
        for p in os.listdir(manifesto_dir)
        if p.startswith(party)
    ]
    
    docs_by_year = {}
    for path in manifesto_paths:
        year = os.path.basename(path).split("_")[-1].replace(".txt", "")
        with open(path, "r", encoding="utf-8") as f:
            text = f.read()
        docs_by_year[year] = splitter.create_documents([text])

    vectorstores = {}
    for year, docs in docs_by_year.items():
        store_path = os.path.join(vs_root, f"{party}_{year}")
        if os.path.exists(store_path):
            vectorstores[year] = Chroma(
                persist_directory=store_path,
                embedding_function=embeddings
            )
        else:
            vs = Chroma.from_documents(
                docs,
                embeddings,
                persist_directory=store_path
            )
            vectorstores[year] = vs
    return vectorstores




vote_embeddings = pd.read_parquet("data/parquet/votes_summarized_embeddings.parquet")
vote_embeddings["date"] = pd.to_datetime(vote_embeddings["vote"].str.split("_").str[0], format="%Y%m%d")
manifestos_metadata = pd.read_csv("data/csv/manifestos.csv")
manifestos_metadata["valid_starting"] = pd.to_datetime(manifestos_metadata["valid_starting"], format="%d.%m.%Y")


def get_correct_manifesto_year(date: datetime, party: str) -> str | None:
    only_party = manifestos_metadata.query("party == @party").sort_values("valid_starting", ascending=True)
    after_date = only_party.query("valid_starting < @date")
    try:
        correct_row = after_date.iloc[-1]
        return str(correct_row["year"])
    except IndexError:
        logger.warning(f"No manifesto found for {party} before {date.strftime('%Y-%m-%d')}")
        return None
    



DEEPSEEK_SYSTEM_PROMPT = """
    Entscheide anhand der folgenden Informationen aus dem Wahlprogramm einer imaginären Partei, ob die Partei sich bei dem gegebenen Antrag enthalten hat, oder für bzw. gegen den Antrag im Bundestag gestimmt hat.
    Der Output muss immer mit entweder "enthält sich", "stimmt zu" oder "stimmt nicht zu" anfangen. mit einer kurzen Begründung wie du zu der Entscheidung gekommen bist.
"""

def predict_vote(vote: pd.Series, chroma_store: dict[str, Chroma], party: str):
    relevant_year = get_correct_manifesto_year(vote["date"], party)
    if relevant_year is None:
        return None
    vs = chroma_store[relevant_year]
    results = vs.similarity_search_by_vector(
        embedding=vote["embedding"],
        k=5
    )
    
    llm_context = "\n".join([doc.page_content for doc in results])
    decision_text = prompt_deepseek(
        system_prompt=DEEPSEEK_SYSTEM_PROMPT,
        text=f"""
            Wahlprogramm: {llm_context} 
            Antrag: {vote["vote"]}
        """
    )
    cleaned = re.sub(r'[^a-zA-Z ]', '', decision_text).strip()
    if cleaned.startswith('stimmt nicht zu'):
        decision = "ablehnung"
    elif cleaned.startswith('stimmt zu'):
        decision = "zustimmung"
    elif cleaned.startswith('enthält sich'):
        decision = "enthaltung"
    else:
        logger.warning(f"Unexpected decision text: {decision_text}")
        decision = None
    return {
        "context": llm_context,
        "reasoning": decision_text,
        "decision": decision
    }


def process(idx: int, row: pd.Series, chroma_store: dict[str, Chroma], party: str) -> tuple[int, str | None]:
    try:
        return idx, predict_vote(row, chroma_store, party)
    except Exception as e:
        logger.error(f"Error processing row: {row['vote']}")
        logger.exception(e)
        return idx, None


def predict_partyline(party: str) -> list[dict]:
    decisions = [None] * len(vote_embeddings)
    manifesto_embeddings = embed_manifestos(party) 
    with ThreadPoolExecutor(max_workers=8) as pool, tqdm(total=len(vote_embeddings)) as pbar:
        futures = [
            pool.submit(process, i, row, manifesto_embeddings, party)
            for i, (_, row) in enumerate(vote_embeddings.iterrows())
        ]
        for future in as_completed(futures):
            i, decision = future.result()
            decisions[i] = decision
            pbar.update(1)
    return decisions


for party in ["AfD", "DIE_GRÜNEN", "DIE_LINKE", "FDP", "SPD", "Union"]:
    logger.info(f"Processing party: {party}")
    party_lines = predict_partyline(party)
    vote_embeddings[f"{party}_decision"] = party_lines

2025-05-24 18:30:54.832 | INFO     | __main__:<module>:65 - Processing party: AfD
2025-05-24 18:30:54.833 | INFO     | __main__:embed_manifestos:23 - Embedding manifestos for AfD
 34%|███▎      | 210/623 [04:27<06:03,  1.14it/s]2025-05-24 18:35:22.267 | WARNING  | __main__:get_correct_manifesto_year:75 - No manifesto found for AfD before 2013-02-28
2025-05-24 18:35:22.270 | WARNING  | __main__:get_correct_manifesto_year:75 - No manifesto found for AfD before 2013-02-28
2025-05-24 18:35:22.272 | WARNING  | __main__:get_correct_manifesto_year:75 - No manifesto found for AfD before 2013-02-28
2025-05-24 18:35:22.275 | WARNING  | __main__:get_correct_manifesto_year:75 - No manifesto found for AfD before 2013-02-28
2025-05-24 18:35:22.277 | WARNING  | __main__:get_correct_manifesto_year:75 - No manifesto found for AfD before 2013-02-28
2025-05-24 18:35:22.280 | WARNING  | __main__:get_correct_manifesto_year:75 - No manifesto found for AfD before 2013-01-31
2025-05-24 18:35:22.282 | WARNING 

In [8]:
vote_embeddings.sample(1)["DIE_GRÜNEN_decision"].iloc[0]

{'context': '- Reform und Stärkung der Europäischen Union zu einer demokratischen Wertegemeinschaft mit eigener Fiskal- und sozialpolitischer Handlungsfähigkeit, Stärkung des Europaparlaments, Einführung transnationaler Listen, und Auflösung blockierender Einstimmigkeitsregelungen.\n- Globale Sicherheitspolitik mit Schwerpunkt auf multilateraler Krisenprävention, ziviler Konfliktbearbeitung, Schutz der Menschenrechte weltweit, Atomwaffenver\xadbot, und Rüstungskontrollabkommen.\n- Entwicklung einer modernen, zukunftsfähigen Bundeswehr, transparente und verantwortungsvolle Sicherheitspolitik in europäischen und transatlantischen Bündnissen.\n- Umsetzung einer menschrechtsorientierten Einwanderungspolitik mit Einbürgerungserleichterungen, Sprachförderung, Rechteausbau und Verbesserung der Lage von Geflüchteten.\n- Sicherstellung fairer und effizienter Asylverfahren, Abschaffung des Asylbewer\xad berleistungsgesetzes, Schutz von Geflüchteten, Sicherheit und faire Verteilung in Europa durc